## The first big project - Professionally You!

### And, Tool use.

### But first: introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Remember to save your `.env` file, and run `load_dotenv(override=True)` after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [23]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

import os
from IPython.display import Markdown, display
from limits import parse
from limits.storage import RedisStorage
from limits.strategies import FixedWindowRateLimiter
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [24]:
# The usual start

load_dotenv(override=True)
google_api_key = os.getenv('GEMINI_API_KEY')

In [25]:
MODEL_LIMITS = {
    # --- Gemini 2.5 Series ---
    "gemini-2.5-pro": {"rpd": 50, "rpm": 2, "tpm": 125000, "tpd":3000000},
    # "gemini-2.5-pro-1p-freebie": {"rpd": 500, "rpm": 750, "tpm": 1000000},
    "gemini-2.5-flash": {"rpd": 250, "rpm": 10, "tpm": 250000, "tpd":None},
    "gemini-2.5-flash-lite": {"rpd": 1000, "rpm": 15, "tpm": 250000, "tpd":None}, # can
    "gemini-2.5-flash-live": {"rpd": None, "rpm": None, "tpm": 1000000, "tpd":None},
    # "gemini-2.5-flash-tts": {"rpd": 150, "rpm": 30, "tpm": 10000},
    # "gemini-2.5-flash-native-audio-dialog": {"rpd": 50, "rpm": None, "tpm": 25000},
    # "gemini-2.5-flash-exp-native-audio-thinking-dialog": {"rpd": 50, "rpm": None, "tpm": 10000},

    # # --- Gemini 2.0 Series ---
    # "gemini-2.0-flash": {"rpd": 200, "rpm": 150, "tpm": 1000000},
    # "gemini-2.0-flash-lite": {"rpd": 2000, "rpm": 300, "tpm": 1000000},
    # "gemini-2.0-flash-live": {"rpd": None, "rpm": None, "tpm": 1000000},
    # "gemini-2.0-exp": {"rpd": 500, "rpm": 50, "tpm": 1000000},

    # # --- Gemini 1.5 Series ---
    # "gemini-1.5-flash": {"rpd": 500, "rpm": 150, "tpm": 250000},
    # "gemini-1.5-flash-8b": {"rpd": 500, "rpm": 150, "tpm": 250000},

    # # --- Gemma 3 Series ---
    # "gemma-3-1b": {"rpd": 14400, "rpm": 300, "tpm": 15000},
    # "gemma-3-2b": {"rpd": 14400, "rpm": 300, "tpm": 15000},
    # "gemma-3-4b": {"rpd": 14400, "rpm": 300, "tpm": 15000},
    # "gemma-3-12b": {"rpd": 14400, "rpm": 300, "tpm": 15000},
    # "gemma-3-27b": {"rpd": 14400,"rpm": 300,"tpm": 15000},
    # # --- Other & Experimental Models ---
    # "chat-bard": {"rpd": 2520000, "rpm": 18000, "tpm": None},
    # "computer-use-exp": {"rpd": None, "rpm": 1000, "tpm": None},
    # "gqi-cst-h34jgm": {"rpd": None, "rpm": 500, "tpm": 3000000},
    # "gemini-1.0-pro": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-1.5-pro": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-1.5-pro-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-1.5-flash-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-1.5-flash-8b-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-2.0-pro-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-2.5-pro-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-2.5-pro-tts": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-2.5-flash-preview-image": {"rpd": 0, "rpm": 0, "tpm": 0},
}

def callLLmApi(api_key, base_url, model, messages, tools=None, expected_prompt_tokens=None, expected_completion_tokens=None, response_format=None):
    def _estimate_tokens(msgs):
        try:
            txt = "".join(m.get("content", "") for m in msgs if isinstance(m, dict))
            return max(1, len(txt) // 4)  # rough 4 chars/token fallback
        except Exception:
            return 0

    limits = MODEL_LIMITS.get(model, {})

    # Ensure a limiter storage is available (prefer Redis, fallback to in-memory)
    storage_obj = globals().get("storage")
    if storage_obj is None:
        try:
            storage_obj = RedisStorage.from_url(os.getenv("REDIS_URL", "redis://localhost:6379"))
        except Exception:
            try:
                from limits.storage import MemoryStorage
                storage_obj = MemoryStorage()
            except Exception:
                storage_obj = None
        globals()["storage"] = storage_obj

    limiter = FixedWindowRateLimiter(storage_obj) if storage_obj else FixedWindowRateLimiter(RedisStorage.from_url("redis://localhost:6379"))

    rpm = limits.get("rpm")
    rpd = limits.get("rpd")
    tpm = limits.get("tpm")
    tpd = limits.get("tpd")

    # Keys per model
    req_key = f"api:req:{model}"
    tpd_key = f"api:tok:day:{model}"
    tpm_key = f"api:tok:month:{model}"

    # Time windows
    limit_rpm = parse(f"{rpm}/minute") if rpm else None
    limit_rpd = parse(f"{rpd}/day") if rpd else None
    limit_tpd = parse(f"{tpd}/day") if tpd else None
    limit_tpm = parse(f"{tpm}/month") if tpm else None

    # Predicted token spend (used for pre-check). Fallback to prompt-only estimate if no hints provided.
    if expected_prompt_tokens is not None or expected_completion_tokens is not None:
        expected_total_tokens = (expected_prompt_tokens or 0) + (expected_completion_tokens or 0)
    else:
        expected_total_tokens = _estimate_tokens(messages)

    # Pre-check: ensure this call would not exceed any limits
    if limit_rpm and not limiter.test(limit_rpm, req_key, 1):
        display(Markdown(f"⚠️ Rate limit hit: {rpm} rpm for {model}. Please wait."))
        return None
    if limit_rpd and not limiter.test(limit_rpd, req_key, 1):
        display(Markdown(f"⚠️ Daily request limit hit: {rpd} rpd for {model}."))
        return None
    if limit_tpd and expected_total_tokens and not limiter.test(limit_tpd, tpd_key, expected_total_tokens):
        display(Markdown(f"⚠️ Daily token limit would be exceeded for {model}."))
        return None
    if limit_tpm and expected_total_tokens and not limiter.test(limit_tpm, tpm_key, expected_total_tokens):
        display(Markdown(f"⚠️ Monthly token limit would be exceeded for {model}."))
        return None

    # Reserve request slots (avoids race with other callers)
    if limit_rpm and not limiter.hit(limit_rpm, req_key, 1):
        display(Markdown(f"⚠️ Rate limit hit: {rpm} rpm for {model}. Please wait."))
        return None
    if limit_rpd and not limiter.hit(limit_rpd, req_key, 1):
        display(Markdown(f"⚠️ Daily request limit hit: {rpd} rpd for {model}."))
        return None

    llm = OpenAI(api_key=api_key, base_url=base_url)
    model_name = model

    # Real call (with optional tools)
    if response_format is not None:
        response = llm.chat.completions.parse(
            model=model_name,
            messages=messages,
            response_format=response_format,
            tools=tools
        )
        response_content = None
    else:
        response = llm.chat.completions.create(
            model=model_name,
            messages=messages,
            tools=tools
        )
        response_content = response.choices[0].message.content

    # Try to capture actual token usage if provided by the API
    actual_total_tokens = None
    try:
        usage = getattr(response, "usage", None)
        if isinstance(usage, dict):
            actual_total_tokens = usage.get("total_tokens")
        else:
            actual_total_tokens = getattr(usage, "total_tokens", None)
    except Exception:
        actual_total_tokens = None

    # Token spend accounting (fallback to expected if actual unavailable)
    token_spend = actual_total_tokens if actual_total_tokens is not None else expected_total_tokens

    # Record token usage against daily/monthly quotas
    if token_spend and (limit_tpd or limit_tpm):
        if limit_tpd and not limiter.hit(limit_tpd, tpd_key, token_spend):
            display(Markdown(f"⚠️ Daily token limit exceeded for {model}."))
            return None
        if limit_tpm and not limiter.hit(limit_tpm, tpm_key, token_spend):
            display(Markdown(f"⚠️ Monthly token limit exceeded for {model}."))
            return None

    if response_content is not None:
        display(Markdown(response_content))
    return response

In [50]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [51]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [48]:
push("HEY!!")

Push: HEY!!


I don't have access to your personal information unless you choose to share it with me. My purpose here is to provide information about my own background and experience as Purim Wittayasirikul.

If you'd like to share your name or any other details you're comfortable with, I can record them. Otherwise, we can continue our conversation focusing on my expertise as a Software Engineer!

Yes, that's exactly right! If I encounter a question that I don't know the answer to, or if a question falls outside the scope of information I have access to, I'm programmed to use my `record_unknown_question` tool. This helps to ensure that I can keep track of what I don't know and potentially improve my knowledge base over time.

Do you have any questions for me about my skills or experience as a Software Engineer? I'd be happy to answer them!

In [52]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

Thank you for providing your email address! I've recorded it. If you'd like to get in touch directly or have any further questions, feel free to use the contact information available on my website. I'm always open to discussing potential collaborations or opportunities!

In [30]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [53]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [54]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [55]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [56]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

In [ ]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

# def handle_tool_calls(tool_calls):
#     results = []
#     for tool_call in tool_calls:
#         tool_name = tool_call.function.name
#         arguments = json.loads(tool_call.function.arguments)
#         print(f"Tool called: {tool_name}", flush=True)

#         # THE BIG IF STATEMENT!!!

#         if tool_name == "record_user_details":
#             result = record_user_details(**arguments)
#         elif tool_name == "record_unknown_question":
#             result = record_unknown_question(**arguments)

#         results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
#     return results

In [58]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [67]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [61]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Purim Wittayasirikul"

In [68]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

system_prompt


"You are acting as Purim Wittayasirikul. You are answering questions on Purim Wittayasirikul's website, particularly questions related to Purim Wittayasirikul's career, background, skills and experience. Your responsibility is to represent Purim Wittayasirikul for interactions on the website as faithfully as possible. You are given a summary of Purim Wittayasirikul's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. \n\n## Summary:\nMy name is Purim Wittayasirikul, my Nick name is CD. I'm an Software E

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = callLLmApi(
        google_api_key,
        "https://generativelanguage.googleapis.com/v1beta/openai/",
        "gemini-2.5-flash-lite",
        messages,
        tools
    )
        
        print(response.choices[0].finish_reason)
        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [70]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Hi there! It's great to connect with you. How can I help you today? Are you interested in learning more about my software engineering background, my experience with specific technologies, or perhaps something else?

stop


As a software engineer, I don't have a daughter. Is there anything else I can help you with regarding my professional background or experience?

stop


...

stop


I can help with that! To use the tool to record contact information, I need a bit more from you. Could you please provide your email address? If you'd also like to share your name or any specific notes, I can record those too.

stop
tool_calls
Tool called: record_user_details
Push: Recording interest from Name not provided with email aa@gmail.com and notes not provided


Thank you! I've recorded your contact information. It was great chatting with you. If there's anything else you need, feel free to ask!

stop


## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you! Also change `self.name = "Ed Donner"` in `app.py`..  

Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions - it needs to have WRITE permissions! Keep a record of your new key.  
3. In the Terminal, run: `uv tool install 'huggingface_hub[cli]'` to install the HuggingFace tool, then `hf auth login` to login at the command line with your key. Afterwards, run `hf auth whoami` to check you're logged in  
4. Take your new token and add it to your .env file: `HF_TOKEN=hf_xxx` for the future
5. From the 1_foundations folder, enter: `uv run gradio deploy` 
6. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

Thank you Robert, James, Martins, Andras and Priya for these tips.  
Please read the next 2 sections - how to change your Secrets, and how to redeploy your Space (you may need to delete the README.md that gets created in this 1_foundations directory).

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets (Variables and Secrets section), delete the space, etc.

#### And now you should be deployed!

If you want to completely replace everything and start again with your keys, you may need to delete the README.md that got created in this 1_foundations folder.

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• First and foremost, deploy this for yourself! It's a real, valuable tool - the future resume..<br/>
            • Next, improve the resources - add better context about yourself. If you know RAG, then add a knowledge base about you.<br/>
            • Add in more tools! You could have a SQL database with common Q&A that the LLM could read and write from?<br/>
            • Bring in the Evaluator from the last lab, and add other Agentic patterns.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>